In [1]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("FEM.jl")
using .FEM

In [2]:
# Adatok
E = 2e5
ν = 0.3
thickness = 1

problem = FEM.Problem("Lshape", type="PlaneStress", E=E, ν=ν, b=thickness)

Main.FEM.Problem("Lshape", "PlaneStress", 200000.0, 0.3, 7.85e-9, 1.0)

In [3]:
gmsh.merge("LshapedPlate.geo")
gmsh.merge("LshapedPlate.msh")

Info    : Reading 'LshapedPlate.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Line)
Info    : [ 90%] Meshing curve 6 (Line)
Info    : Done meshing 1D (Wall 0.00503769s, CPU 0.004037s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00462854s, CPU 0.004626s)
Info    : Meshing order 4 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 4
Info    : [ 20%] Meshing curve 2 order 4
Info    : [ 30%] Meshing curve 3 order 4
Info    : [ 50%] Meshing curve 4 order 4
Info    : [ 60%] Meshing curve 5 order 4
Info    : [ 80%] Meshing curve 6 order 4
Info    : [ 90%] Meshing surface 1 order 4
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.770618
Info    : Done meshing order 4 (Wall 0.00412019s, CPU 0.004115s)
Info

In [4]:
bc1 = FEM.displacementConstraint("fix", ux=0, uy=0)
ld1 = FEM.traction("load", fy=1)

("load", 0, 1, 1)

In [5]:
#gmsh.fltk.run()

In [6]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [7]:
K = FEM.stiffnessMatrixPlaneStress(problem)

5242×5242 SparseMatrixCSC{Float64, Int64} with 239844 stored entries:
⎡⢿⣷⡛⣷⣿⡉⠨⠐⠄⣤⠢⢠⠰⠶⣖⠾⠆⠀⢶⣦⣿⣌⣑⢼⣯⡋⢇⣤⠒⢚⠟⣶⢔⢈⣁⡒⠀⢀⡐⠂⎤
⎢⢿⣬⣿⣿⣿⣼⣤⣧⣴⣴⣤⣽⣦⣶⢯⢽⣴⣶⣾⣬⣥⣭⣥⣬⣦⣥⣤⣄⢷⣬⣤⣤⣤⣬⣤⣥⣤⣤⣤⣤⎥
⎢⡟⠻⣛⣿⣿⣿⡿⡿⡓⠟⣟⠛⠛⠛⣛⡙⠻⠟⢛⠟⡻⢿⡿⠫⠟⠻⣿⠿⠟⠟⢻⠿⠻⠻⠿⠿⠻⠿⠿⠟⎥
⎢⢂⠂⠤⣿⣿⡯⣿⣿⣷⡤⣛⡀⣷⡅⠌⠄⠠⠠⠠⠀⠀⠈⠒⠈⠐⠀⢁⠀⠌⠑⠀⠈⢀⠡⠄⠂⢁⠀⠆⠌⎥
⎢⠀⣥⢐⣿⣽⠌⠙⡿⢿⣷⡑⠢⠆⣋⠁⠶⢔⣀⣜⣁⠀⢀⡄⡀⢄⢀⠂⠀⠀⠐⠠⠀⠀⠀⠀⠠⠄⠠⠤⢒⎥
⎢⠈⣂⣄⣿⣿⠙⠛⠸⠱⡈⢻⣶⡧⣈⣤⡌⠞⠤⡀⠆⠐⠁⠁⣒⣢⠓⠘⠀⠠⠋⠂⣁⠀⠀⠁⠙⠀⠀⠁⠀⎥
⎢⢰⡆⢨⣿⣿⠀⠝⠿⡬⢡⡉⢫⡿⣯⡿⣟⠂⠠⣁⠀⠀⠀⠀⠀⠒⡀⢓⠇⠀⠠⢄⠀⠠⠅⠑⠑⣀⠀⠁⠀⎥
⎢⣸⡝⣏⣗⣟⠸⠂⠅⢡⡄⡀⠿⣿⢯⣿⣿⡘⡂⠈⡄⢡⠙⠀⢆⠰⠀⠀⠄⡑⠂⠐⠂⠀⠂⠀⡀⠀⠐⡀⠀⎥
⎢⠈⠁⢰⣿⣿⠆⠀⡂⠐⢱⠚⡅⠈⡀⠲⠨⣻⣾⣀⠀⠁⠦⠠⠅⠆⡡⢒⢀⣦⠆⠐⢢⠹⡦⡓⠃⠙⠙⠀⠀⎥
⎢⠸⣷⡚⣿⣿⠔⠀⠂⠖⢹⠠⠌⠁⠘⠂⠤⠀⠘⢻⣶⡔⣊⠠⣦⢠⡈⢁⡉⠊⠁⡠⠵⠆⢁⣀⠀⠐⠂⠂⠣⎥
⎢⡛⢿⡅⣿⣿⣎⡀⠀⠀⢀⠔⠀⠀⠀⣅⠒⠡⡄⡰⢩⡿⣯⡅⠘⠿⢈⡠⠀⠊⡘⠛⢄⠀⢀⠠⠀⠀⡈⠁⠀⎥
⎢⣑⣜⡁⣿⡿⡋⡘⠀⠀⠩⢡⢠⠀⠀⠠⢄⠄⠆⠠⣦⣁⠉⣻⣾⡨⠋⢔⡑⠒⢻⡇⡈⣐⢥⡀⠊⣸⡕⠚⠂⎥
⎢⡯⠻⠌⣿⣿⡁⠐⠀⠀⢑⢬⠚⠘⠠⠐⠂⠌⡡⡀⠲⡛⢃⡦⠊⢿⣷⣻⠉⢡⠔⣂⣰⢤⣈⡤⠄⠨⠩⠈⠐⎥
⎢⠉⣵⠀⢿⣿⡟⠁⠐⠈⠀⠒⠀⠽⠔⠀⠄⠘⢐⡅⠰⠀⠊⢔⠱⡟⠚⢻⣶⢀⡜⡝⠍⡔⡭⢧⠒⣐⣣⠔⡉⎥
⎢⣸⢀⡙⣷⣿⠅⢆⠁⢀⠀⡤⠂⠀⡀⠱⠈⠨⠟⠎⠀⣊⠠⣼⣀⢁⠖⣀⠴⣿⣿⣅⠰⢰⡶⢨⡄⠂⢌⣊⣃⎥
⎢⢻⣥⠀⣿⣿⡖⡀⠀⠀⠂⠌⢠⠀⠑⠰⠀⠰⣀⢄⡎⠛⢄⡉⠩⢈⣸⡗⠍⢁⡙⠻⣦⣾⠡⢴⡸⢢⡐⠊⢇⎥
⎢⡐⢑⡀⣿⣿⡂⠄⡐⠀⠀⠀⠀⠄⠆⠠⠀⠳⡦⠌⢁⠀⢀⠔⣜⡀⢳⡔⡭⢰⡶⠞⡛⢻⣶⡋⣭⢵⡥⡩⡀⎥
⎢⢡⠸⠄⣿⣿⡇⠠⠁⠀⡀⣅⠀⢕⠀⠀⠠⠽⠈⠀⠘⠀⠂⡠⠈⠀⠏⢩⠓⠂⠶⣐⡳⡏⣬⢿⣷⡨⡿⢿⡂⎥
⎢⠀⢀⠀⣿⣿⡆⠁⠐⠀⡁⠀⠀⠀⠘⢀⠀⣗⠀⠰⠀⡀⠠⢖⠾⡆⡂⠴⣸⡈⢄⢈⠲⠕⡷⣦⡮⡿⣯⣭⣦⎥
⎣⠰⠈⠀⣿⣿⠇⡈⠅⢠⢃⠁⠀⠁⠀⠀⠈⠀⠀⠬⡀⠁⠀⠺⠀⢂⠀⡔⠡⠮⢸⠮⢄⠃⠪⠻⠳⠣⣿⡿⣯⎦

In [8]:
K, f = FEM.applyBoundaryConditions!(problem, K, [bc1], [ld1])

(sparse([1, 2, 9, 10, 15, 16, 17, 18, 19, 20  …  4171, 4172, 4173, 4174, 5237, 5238, 5239, 5240, 5241, 5242], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  5242, 5242, 5242, 5242, 5242, 5242, 5242, 5242, 5242, 5242], [179624.78643730748, -82399.32202686464, -2849.6072665445804, 3053.3146608628713, 62433.81659526875, -54691.94161249685, -13668.087998622741, 18713.50900390918, 2482.713681353929, -6728.535568748624  …  -416721.6686121728, -729158.0516223616, 114250.45193944963, 232982.29765051874, 400124.5533570185, -565086.4826328191, -441858.82143113395, -959824.470821564, 510111.5267111789, 2.756243278705254e6], 5242, 5242), [0.0; 3.8888888888739492; … ; 0.0; 0.0;;])

In [9]:
#det(K)

In [10]:
q = FEM.solveDisplacement(K, f)

5242×1 Matrix{Float64}:
 0.01976161257386661
 0.024378452709530655
 0.002027928336528375
 0.024419746278511525
 0.0
 0.0
 0.0
 0.0
 0.015160227744530255
 0.024204421762142464
 ⋮
 0.008334024617587508
 0.004221504726377839
 0.008276791154461892
 0.00410576056905915
 0.005834310408455397
 0.00400273507531214
 0.005834516427547291
 0.004032963511248456
 0.0057225362521883955

In [11]:
S = FEM.solveStressPlaneStress(problem, q)

"elemTags = Vector{UInt64}[[0x0000000000000035, 0x0000000000000036, 0x0000000000000037, 0x0000000000000038, 0x0000000000000039, 0x000000000000003a, 0x000000000000003b, 0x000000000000003c, 0x000000000000003d, 0x000000000000003e, 0x000000000000003f, 0x0000000000000040, 0x0" ⋯ 12128 bytes ⋯ "0000163, 0x0000000000000164, 0x0000000000000165, 0x0000000000000166, 0x0000000000000167, 0x0000000000000168, 0x0000000000000169, 0x000000000000016a, 0x000000000000016b, 0x000000000000016c, 0x000000000000016d, 0x000000000000016e, 0x000000000000016f, 0x0000000000000170]]"

([[-0.5544963824165032, 1.7047230513753322, 0.0, 1.7047230513753322, -21.54843587150753, 0.0, 0.0, 0.0, 0.0, -2.1581319453466152  …  0.0, -0.7022844594760391, 2.2039235481341697, 0.0, 2.2039235481341697, -23.277830436487076, 0.0, 0.0, 0.0, 0.0], [-2.37270054971646, -0.028136350901311787, 0.0, -0.028136350901311787, -4.64590654412532, 0.0, 0.0, 0.0, 0.0, -4.094914484470317  …  0.0, -2.5609748500536083, 0.8145435398725638, 0.0, 0.8145435398725638, -8.429960051476948, 0.0, 0.0, 0.0, 0.0], [-0.0006593804710190582, 1.998581993618707, 0.0, 1.998581993618707, 0.3280477244185696, 0.0, 0.0, 0.0, 0.0, -0.5964567425976541  …  0.0, -0.21626559362800868, 2.9918466417167013, 0.0, 2.9918466417167013, 0.2507414021920692, 0.0, 0.0, 0.0, 0.0], [-0.1284892578361072, 0.2986446639068745, 0.0, 0.2986446639068745, -16.86671225602379, 0.0, 0.0, 0.0, 0.0, -5.919413078725455e-5  …  0.0, -0.05289906051986293, 0.1388961364170466, 0.0, 0.1388961364170466, -16.556756859319353, 0.0, 0.0, 0.0, 0.0], [-2.3762966596564

In [12]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [13]:
u = FEM.showResultUvec(problem, q, name="uvec", visible=false)
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
sx = FEM.showResultSX(problem, S, name="σx", visible=false, smooth=false)
sy = FEM.showResultSY(problem, S, name="σy", visible=true, smooth=false)
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)

"numElem = [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 1" ⋯ 2536 bytes ⋯ " 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368]"

632-element Vector{Int64}:
  53
  54
  55
  56
  57
  58
  59
  60
  61
  62
   ⋮
 360
 361
 362
 363
 364
 365
 366
 367
 368

632-element Vector{Any}:
 [-0.5544963824165032, -2.1581319453466152, -0.35220873114177387, -0.7686145567344447, -1.0632358913590716, -1.504396362271745, -1.5576003651937693, -1.0528803636450919, -0.649305349445898, -0.4235673185192441, -0.47790841523107463, -0.5199715826899469, -0.7404743414467818, -1.0643386102615064, -0.7022844594760391]
 [-2.37270054971646, -4.094914484470317, -1.8259078767479606, -2.7123557605931703, -3.103738637254714, -3.5601559538452507, -3.499053531446624, -2.899473015725757, -2.330361582475268, -2.093157043441582, -2.2681284372296977, -2.3586877190153515, -2.685684355788039, -3.0613707018807874, -2.5609748500536083]
 [-0.0006593804710190582, -0.5964567425976541, -0.3035443276726288, -0.01258822757899064, -0.0851363045187826, -0.26439525995774915, -0.5627724820199633, -0.505953182860026, -0.42115755981294434, -0.15679404866743205, -0.06357625781917309, -0.014621265101727718, -0.08316571289434227, -0.2505189023167408, -0.21626559362800868]
 [-0.1284892578361072,

7

In [14]:
#gmsh.model.setCurrent(problem.name)
#pp1 = gmsh.model.occ.addPoint(0, 0, 0)
#pp2 = gmsh.model.occ.addPoint(50, 50, 0)

#pl1 = gmsh.model.occ.addLine(pp1, pp2)


#gmsh.model.occ.synchronize()

In [15]:
FEM.plotOnPath(problem, 7, s, 100, name="σy", visible=false);

In [16]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [17]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.1-git-38a28aaaf
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240111
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [18]:
gmsh.finalize()